# Agentes de IA do Azure com Suporte ao Protocolo de Contexto de Modelo (MCP) - Python

Este notebook demonstra como usar os Agentes de IA do Azure com ferramentas do Protocolo de Contexto de Modelo (MCP) em Python. Ele mostra como criar um agente inteligente que pode aproveitar servidores MCP externos (como o Microsoft Learn) para capacidades aprimoradas usando autenticação sem chave.


## Instalar Pacotes Python Necessários

Primeiro, precisamos instalar os pacotes Python necessários:
- **azure-ai-projects**: SDK principal para Projetos de IA do Azure
- **azure-ai-agents**: SDK do Azure AI Agents para criar e gerenciar agentes
- **azure-identity**: Fornece autenticação sem chave usando DefaultAzureCredential
- **mcp**: Implementação do Protocolo de Contexto de Modelo para Python


## Benefícios da Autenticação Sem Chave

Este notebook demonstra a **autenticação sem chave**, que oferece várias vantagens:
- ✅ **Sem necessidade de gerenciar chaves de API** - Utiliza autenticação baseada em identidade do Azure
- ✅ **Segurança aprimorada** - Nenhum segredo armazenado no código ou em arquivos de configuração
- ✅ **Rotação automática de credenciais** - O Azure gerencia o ciclo de vida das credenciais
- ✅ **Controle de acesso baseado em função** - Utiliza o Azure RBAC para permissões detalhadas
- ✅ **Suporte a múltiplos ambientes** - Funciona perfeitamente em desenvolvimento e produção

O `DefaultAzureCredential` seleciona automaticamente a melhor fonte de credenciais disponível:
1. **Identidade Gerenciada** (quando executado no Azure)
2. Credenciais do **Azure CLI** (durante o desenvolvimento local)
3. Credenciais do **Visual Studio**
4. **Variáveis de ambiente** (se configuradas)
5. Autenticação via **navegador interativo** (como alternativa)


## Configuração de Autenticação Sem Chave

**Pré-requisitos para autenticação sem chave:**

### Para Desenvolvimento Local:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Para Ambientes Azure:
- Ative a **Identidade Gerenciada Atribuída pelo Sistema** no seu recurso Azure
- Atribua os **papéis RBAC** apropriados à identidade gerenciada:
  - `Cognitive Services OpenAI User` para acesso ao Azure OpenAI
  - `AI Developer` para acesso a Projetos de IA no Azure

### Variáveis de Ambiente (Opcional):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Sem necessidade de chaves de API ou strings de conexão!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importar Bibliotecas Necessárias

Importe os módulos Python necessários:
- **os, time**: Bibliotecas padrão do Python para variáveis de ambiente e atrasos
- **AIProjectClient**: Cliente principal para Projetos de IA do Azure
- **DefaultAzureCredential**: Autenticação sem chave para serviços do Azure
- **Classes relacionadas ao MCP**: Para criar e gerenciar ferramentas MCP e lidar com aprovações


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Configurar as Configurações do Servidor MCP

Configure a configuração do servidor MCP usando variáveis de ambiente com valores padrão de fallback:
- **MCP_SERVER_URL**: A URL do servidor MCP (padrão para a API do Microsoft Learn)
- **MCP_SERVER_LABEL**: Um rótulo para identificar o servidor MCP (padrão para "mslearn")

Essa abordagem permite uma configuração flexível em diferentes ambientes.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Criar Cliente de Projeto Azure AI (Autenticação Sem Chave)

Inicialize o cliente de projeto Azure AI usando **autenticação sem chave**:
- **endpoint**: A URL do endpoint do projeto Azure AI Foundry
- **credential**: Utiliza `DefaultAzureCredential()` para uma autenticação segura e sem chave
- **Sem necessidade de chaves de API**: Descobre e utiliza automaticamente a melhor credencial disponível

**Fluxo de Autenticação:**
1. Verifica Identidade Gerenciada (em ambientes Azure)
2. Recorre às credenciais do Azure CLI (para desenvolvimento local)
3. Utiliza outras fontes de credenciais disponíveis, conforme necessário

Essa abordagem elimina a necessidade de gerenciar chaves de API ou strings de conexão no seu código.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Criar Definição de Ferramenta MCP

Crie uma ferramenta MCP que se conecte ao servidor MCP do Microsoft Learn:
- **server_label**: Identificador para o servidor MCP
- **server_url**: Endpoint URL do servidor MCP
- **allowed_tools**: Lista opcional para restringir quais ferramentas podem ser usadas (lista vazia permite todas as ferramentas)

Essa ferramenta permitirá que o agente acesse a documentação e os recursos do Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Criar Agente e Executar Conversa (Fluxo Sem Chave)

Esta seção abrangente demonstra o fluxo completo do **agente sem chave**:

1. **Criar Agente de IA**: Configure um agente com o modelo GPT-4.1 nano e ferramentas MCP  
2. **Criar Tópico**: Estabeleça um tópico de conversa para comunicação  
3. **Enviar Mensagem**: Pergunte ao agente sobre as diferenças entre Azure OpenAI e OpenAI  
4. **Gerenciar Aprovações de Ferramentas**: Aprove automaticamente chamadas de ferramentas MCP quando necessário  
5. **Monitorar Execução**: Acompanhe o progresso do agente e gerencie quaisquer ações necessárias  
6. **Exibir Resultados**: Mostre os detalhes da conversa e do uso das ferramentas  

**Recursos Sem Chave:**
- ✅ **Sem segredos hardcoded** - Toda autenticação é gerenciada pela identidade do Azure  
- ✅ **Seguro por padrão** - Utiliza controle de acesso baseado em funções  
- ✅ **Implantação simplificada** - Não é necessário gerenciar credenciais  
- ✅ **Amigável para auditoria** - Todo acesso é rastreado por meio da identidade do Azure  

O agente usará ferramentas MCP para acessar recursos do Microsoft Learn com total segurança e sem necessidade de gerenciamento de chaves de API.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autoritativa. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
